In [3]:
from torch.utils.data import DataLoader,IterableDataset
from datasets import load_dataset

from transformers import AutoModel, AutoTokenizer
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
from torch import nn
from copy import deepcopy
import numpy as np
import gc
%load_ext autoreload
%autoreload 2
    

import sys

src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)


from utils import convert_to_Qwen2_ND, detect_domain_specific_neurons,impacts_on, Collator

In [4]:
qwen_path = 'Qwen/Qwen2.5-7B'

qwen = AutoModel.from_pretrained(qwen_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(qwen_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
qwen = convert_to_Qwen2_ND(qwen).to("cuda")
qwen.eval();

In [8]:
collator = Collator(tokenizer)
dataset = load_dataset('yhavinga/ccmatrix', "en-ru", split='train', streaming=True)
data_loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    collate_fn = collator,
    batch_sampler=None 
)

In [ ]:
qwen, output = detect_domain_specific_neurons(
    qwen,
    tokenizer,
    dataloader=data_loader,
    eps=1e-2,
    domain_name="eng", 
    reset_impacts=False,
    reset_dsn=True,
    num_elements=5
)